In [1]:
import pandas as pd
import numpy as np
import warnings
from pandas.errors import PerformanceWarning
# In our case, performance isn't affected by much, so let's suppress the warning
warnings.simplefilter(action='ignore', category=PerformanceWarning)

# Load the survey data
df_survey = pd.read_csv('2) responses_to_selected_survey_questions_filtered.csv')

### We will first process the demographic information (Q18-34) and perform consistency checks

In [2]:
# Function to determine most common age range
def most_common_age_range(df):
    all_ages = [age for row in df[age_columns].values for age in row if pd.notna(age)]
    return max(set(all_ages), key=all_ages.count) if all_ages else np.nan

# Function to count age groups
def count_age_group(row, age_ranges):
    return sum(1 for age in row if age in age_ranges)

# Process Q18: household size, Q19-26: Gender, Q27-34: Age
df_survey['household_size'] = pd.to_numeric(df_survey['Q18'], errors='coerce')
gender_columns = [f'Q{i}' for i in range(19, 27)]
age_columns = [f'Q{i}' for i in range(27, 35)]

df_survey['male_occupants'] = df_survey[gender_columns].apply(lambda row: (row == 'Male').sum(), axis=1)
df_survey['female_occupants'] = df_survey[gender_columns].apply(lambda row: (row == 'Female').sum(), axis=1)

child_age_ranges = ['0-4', '5-11', '11-May', '12-15', '15-Dec', '16-17']
adult_age_ranges = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75 or older']

df_survey['count_children'] = df_survey[age_columns].apply(lambda row: count_age_group(row, child_age_ranges), axis=1)
df_survey['count_adult'] = df_survey[age_columns].apply(lambda row: count_age_group(row, adult_age_ranges), axis=1)

In [3]:
# Function to perform consistency checks for demographics information
def consistency_checks(df):
    checks = {
        "size != sum of male and female occupants": df['household_size'] != (df['male_occupants'] + df['female_occupants']),
        "size != sum of children and adults": df['household_size'] != (df['count_children'] + df['count_adult']),
        "sum of genders != sum of age groups": (df['male_occupants'] + df['female_occupants']) != (df['count_children'] + df['count_adult']),
        "no adults": df['count_adult'] == 0,
        "more than 8 members": df['household_size'] > 8,
        "more children than total size": df['count_children'] > df['household_size'],
        "all members are children": (df['count_children'] == df['household_size']) & (df['household_size'] > 0)
    }
    
    print("Consistency Checks:")
    for description, condition in checks.items():
        inconsistent = df[condition]
        print(f"Households where {description}: {len(inconsistent)}")
    
    all_inconsistent = pd.concat([df[condition] for condition in checks.values()]).drop_duplicates()
    print("\nDetailed information for inconsistent households:")
    print(all_inconsistent[['Household_id', 'household_size', 'male_occupants', 'female_occupants', 'count_children', 'count_adult']])

# Run initial consistency checks
consistency_checks(df_survey)

Consistency Checks:
Households where size != sum of male and female occupants: 45
Households where size != sum of children and adults: 33
Households where sum of genders != sum of age groups: 32
Households where no adults: 7
Households where more than 8 members: 0
Households where more children than total size: 0
Households where all members are children: 1

Detailed information for inconsistent households:
    Household_id  household_size  male_occupants  female_occupants  \
6          D0010             NaN               0                 0   
14         D0020             NaN               0                 1   
26         D0038             2.0               3                 1   
47         D0086             5.0               3                 1   
57         D0104             NaN               1                 1   
58         D0105             NaN               0                 0   
80         D0143             3.0               1                 1   
107        D0191             

### We will correct the inconsistencies based on available information

In [4]:
# Calculate the proportion of male participants for later imputation
male_proportion = df_survey['male_occupants'].sum() / (df_survey['male_occupants'].sum() + df_survey['female_occupants'].sum())

# Get most common age range
most_common_age = most_common_age_range(df_survey)

# Function to correct demographics information based on available information
def correct_demographics(row, male_prop):
    size = row['household_size']
    gender_count = row['male_occupants'] + row['female_occupants']
    age_count = row['count_children'] + row['count_adult']
    
    # If all counts are consistent, return the row as is
    if pd.notna(size) and size == gender_count == age_count:
        return row
    
    # Impute missing values based on available data
    if pd.notna(size):
        if gender_count == 0 and age_count == 0:
            # If both gender and age counts are missing, estimate them based on household size
            row['male_occupants'] = round(size * male_prop)
            row['female_occupants'] = size - row['male_occupants']
            row['count_adult'] = size if most_common_age in adult_age_ranges else 0
            row['count_children'] = size - row['count_adult']
        elif age_count == 0:
            # If age count is missing, estimate it based on household size
            row['count_adult'] = size if most_common_age in adult_age_ranges else 0
            row['count_children'] = size - row['count_adult']
        elif gender_count == 0:
            # If gender count is missing, estimate it based on household size
            row['male_occupants'] = round(size * male_prop)
            row['female_occupants'] = size - row['male_occupants']
    elif pd.isna(size):
        if gender_count > 0 and age_count == 0:
            # If household size and age count are missing, estimate them based on gender count
            row['household_size'] = gender_count
            row['count_adult'] = gender_count if most_common_age in adult_age_ranges else 0
            row['count_children'] = gender_count - row['count_adult']
        elif gender_count == 0 and age_count > 0:
            # If household size and gender count are missing, estimate them based on age count
            row['household_size'] = age_count
            row['male_occupants'] = round(age_count * male_prop)
            row['female_occupants'] = age_count - row['male_occupants']
        elif gender_count > 0 and age_count > 0:
            # If household size is missing, estimate it based on the maximum of gender and age counts
            row['household_size'] = max(gender_count, age_count)
    
    # Ensure counts do not exceed household size
    row['household_size'] = max(row['household_size'], gender_count, age_count)
    row['male_occupants'] = min(row['male_occupants'], row['household_size'])
    row['female_occupants'] = row['household_size'] - row['male_occupants']
    row['count_adult'] = min(row['count_adult'], row['household_size'])
    row['count_children'] = row['household_size'] - row['count_adult']
    
    return row

# Apply demographic corrections to each row
df_survey = df_survey.apply(lambda row: correct_demographics(row, male_proportion), axis=1)

# Function to check if all demographic information is missing
def all_demo_missing(row):
    return (pd.isna(row['household_size']) and 
            all(pd.isna(row[col]) for col in gender_columns) and 
            all(pd.isna(row[col]) for col in age_columns))

# Count households where all demographic information is missing
all_missing = df_survey[df_survey.apply(all_demo_missing, axis=1)]
print(f"Number of households where all demographic information is missing: {len(all_missing)}")

# Rerun consistency checks after corrections
print("\nConsistency Checks After Corrections:")
consistency_checks(df_survey)

# Print summary statistics after corrections
print("\nSummary statistics after corrections:")
print(df_survey[['household_size', 'male_occupants', 'female_occupants', 'count_children', 'count_adult']].describe())

Number of households where all demographic information is missing: 0

Consistency Checks After Corrections:
Consistency Checks:
Households where size != sum of male and female occupants: 0
Households where size != sum of children and adults: 0
Households where sum of genders != sum of age groups: 0
Households where no adults: 0
Households where more than 8 members: 0
Households where more children than total size: 0
Households where all members are children: 0

Detailed information for inconsistent households:
Empty DataFrame
Columns: [Household_id, household_size, male_occupants, female_occupants, count_children, count_adult]
Index: []

Summary statistics after corrections:
       household_size  male_occupants  female_occupants  count_children  \
count      608.000000      608.000000        608.000000      608.000000   
mean         1.922697        0.978618          0.944079        0.161184   
std          1.035901        0.747012          0.700786        0.510522   
min          1.0

### We will now process the remaining items (Q35-Q135)

In [5]:
# Initialise merge_columns list and append demographic columns to merge_columns
merge_columns = ['Household_id']
merge_columns.extend(['household_size', 'male_occupants', 'female_occupants', 'count_children', 'count_adult'])

# Q35: Process home ownership status
df_survey['ownership_type'] = np.where(df_survey['Q35'].str.contains('outright|mortgage', case=False, na=False), 'owned',
                               np.where(df_survey['Q35'].str.contains('rent|landlord|local authority', case=False, na=False), 'rented',
                               np.where(df_survey['Q35'].notna(), 'other', np.nan)))

# Impute missing values with the most frequent value
most_frequent_ownership = df_survey['ownership_type'].mode()[0]
df_survey['ownership_type'] = df_survey['ownership_type'].fillna(most_frequent_ownership)

# Create dummy variables for ownership type
for ownership in ['owned', 'rented', 'other']:
    df_survey[f'ownership_{ownership}'] = (df_survey['ownership_type'] == ownership).astype(int)
    merge_columns.append(f'ownership_{ownership}')

# Q36: Process work from home
work_from_home_map = {'Never': 0, 'Occasionally': 1, 'About half the time': 2, 'Most/all weekdays': 3}
df_survey['work_from_home'] = df_survey['Q36'].map(work_from_home_map)
merge_columns.append('work_from_home')

# Q37-Q39: Process housing type
df_survey['housing_type'] = np.where(df_survey['Q37'].notna(), 'house',
                              np.where(df_survey['Q38'].notna(), 'apartment',
                              np.where(df_survey['Q39'].notna(), 'mobile', np.nan)))

# Impute missing values with the most frequent value
most_frequent_housing = df_survey['housing_type'].mode()[0]
df_survey['housing_type'] = df_survey['housing_type'].fillna(most_frequent_housing)

# Create dummy variables for housing type
for housing in ['house', 'apartment', 'mobile']:
    df_survey[f'housing_{housing}'] = (df_survey['housing_type'] == housing).astype(int)
    merge_columns.append(f'housing_{housing}')

# Q40-Q41: Process number of rooms and bedrooms
df_survey['count_rooms'] = pd.to_numeric(df_survey['Q40'], errors='coerce')
df_survey['count_bedrooms'] = pd.to_numeric(df_survey['Q41'], errors='coerce')
merge_columns.extend(['count_rooms', 'count_bedrooms'])

# Q42-Q46: Process insulation
insulation_types = ['Double glazing', 'Roof or loft insulation', 'Wall insulation', 
                    'Floor insulation', 'Hot water tank insulation/lagging']

for i, insulation in enumerate(insulation_types):
    question = f'Q{i+42}'
    col_name = f'insulation_{insulation.lower().replace(" ", "_").replace("/", "_")}'
    df_survey[col_name] = df_survey[question].map({'Yes': 1, 'No': 0, "Don't know": np.nan})
    merge_columns.append(col_name)

# Q47: Process central heating type
df_survey['electric_central_heating'] = df_survey['Q47'].apply(lambda x: 1 if 'Electric' in str(x) else (0 if pd.notna(x) and 'Don\'t know' not in str(x) else np.nan))
merge_columns.append('electric_central_heating')

# Q48: Process central heating usage
heating_controls = [
    'manual_boiler',
    'thermostatic_valves',
    'auto_set_times',
    'auto_temp_control',
    'not_sure'
]

control_phrases = [
    'I switch the heating on manually at the boiler when needed',
    'I control the room temperature using the thermostatic valves on the radiator',
    'The heating switches on and off automatically at set times of the day',
    'The heating is controlled automatically by a thermostatic temperature control',
    "I'm not sure how it is controlled/used"
]

for control, phrase in zip(heating_controls, control_phrases):
    col_name = f'heating_{control}'
    df_survey[col_name] = df_survey['Q48'].str.contains(phrase, na=False).astype(int)
    merge_columns.append(col_name)

# Q49: Process water heating
df_survey['uses_electric_heater'] = df_survey['Q49'].apply(lambda x: 1 if 'electric' in str(x).lower() else (0 if pd.notna(x) and 'Don\'t know' not in str(x) else np.nan))
merge_columns.append('uses_electric_heater')

In [6]:
# Q50-Q79: Process light bulb counts
bulb_types = {
    'low_efficiency': list(range(50, 56)) + list(range(62, 68)),  # Traditional and halogen bulbs
    'high_efficiency': list(range(56, 62)) + list(range(68, 74)),  # Low energy and LED
    'fluorescent_tubes': range(74, 80)
}

# Process each bulb type
for bulb_type, question_range in bulb_types.items():
    if bulb_type == 'fluorescent_tubes':
        col_name = f'count_{bulb_type}'
    else:
        col_name = f'count_{bulb_type}_bulbs'
    # Sum the counts for each bulb type
    df_survey[col_name] = df_survey[[f'Q{i}' for i in question_range]].sum(axis=1)
    merge_columns.append(col_name)

# Q80-93: Process refrigeration appliances
fridge_sizes = [1, 2, 3, 3.5]  # Relative sizes for fridges
freezer_sizes = [2.5, 3.5, 1.5, 2.5, 3.5, 4.5]  # Relative sizes for freezers
fridge_freezer_sizes = [2.25, 3.25, 4.25, 5.25]  # Relative sizes for fridge-freezers

def calculate_total_refrigeration_units(row):
    fridge_total = sum(count * size for count, size in zip([row[f'Q{i}'] for i in range(80, 84)], fridge_sizes))
    freezer_total = sum(count * size for count, size in zip([row[f'Q{i}'] for i in range(84, 90)], freezer_sizes))
    fridge_freezer_total = sum(count * size for count, size in zip([row[f'Q{i}'] for i in range(90, 94)], fridge_freezer_sizes))
    return fridge_total + freezer_total + fridge_freezer_total

df_survey['total_refrigeration_units'] = df_survey.apply(calculate_total_refrigeration_units, axis=1)
merge_columns.append('total_refrigeration_units')

# Also calculate the count of each type
df_survey['fridge_count'] = df_survey[[f'Q{i}' for i in range(80, 84)]].sum(axis=1)
df_survey['freezer_count'] = df_survey[[f'Q{i}' for i in range(84, 90)]].sum(axis=1)
df_survey['fridge_freezer_count'] = df_survey[[f'Q{i}' for i in range(90, 94)]].sum(axis=1)
df_survey['count_fridges_and_freezers'] = df_survey['fridge_count'] + df_survey['freezer_count'] + df_survey['fridge_freezer_count']
merge_columns.append('count_fridges_and_freezers')

# Q94-Q115: Process other appliance counts
appliance_groups = {
    'cooking_appliances': ['Q94', 'Q95', 'Q96', 'Q97'],  # Electric hob, Gas hob, Electric oven, Microwave
    'laundry_appliances': ['Q98', 'Q99', 'Q100'],  # Washing machine, Tumble dryer, Washer-dryer
    'kitchen_appliances': ['Q101'],  # Dishwasher
    'heating_water_appliances': ['Q102', 'Q103', 'Q104'],  # Electric shower, Over-sink electric water heater, Portable electric heater
    'entertainment_devices': ['Q105', 'Q110', 'Q111', 'Q112', 'Q113', 'Q114'],  # TV, DVD player, Cable TV box, Satellite TV box, Freeview TV box, Games console
    'computing_devices': ['Q106', 'Q107', 'Q108', 'Q109'],  # Desktop PC, Laptop, Printer, Router
    'energy_saving_devices': ['Q115']  # Standby savers
}

for group, questions in appliance_groups.items():
    col_name = f'count_{group}'
    df_survey[col_name] = df_survey[questions].sum(axis=1)
    merge_columns.append(col_name)

# Process TV information (Q116-Q127)
def tv_energy_score(tv_type, tv_size):
    type_score = {'Traditional/older style (CRT)': 3, 'LED': 1, 'LCD': 2, 'Plasma': 4, 'Other': 2.5}
    size_score = {'24 inches or less': 1, '25 to 32 inches': 2, '33 to 49 inches': 3, '50 inches or more': 4}
    
    tv_type = str(tv_type) if pd.notna(tv_type) else ''
    tv_size = str(tv_size) if pd.notna(tv_size) else ''
    
    return type_score.get(tv_type, 0) * size_score.get(tv_size, 0)

tv_types = [f'Q{i}' for i in range(116, 122)]
tv_sizes = [f'Q{i}' for i in range(122, 128)]

df_survey['count_tv'] = df_survey[tv_types].notna().sum(axis=1)
df_survey['tv_energy_score'] = sum(df_survey.apply(lambda row: tv_energy_score(row[type_col], row[size_col]), axis=1) 
                                   for type_col, size_col in zip(tv_types, tv_sizes))

merge_columns.extend(['count_tv', 'tv_energy_score'])

In [7]:
# Q128-Q129: Process smart meter views and usage
# Questions about other household members' observed behavior have nearly half the responses missing, so they have been omitted.
smart_meter_questions = ['Q128', 'Q129']

def map_frequency(x):
    mapping = {
        'Never': 0,
        'Less often than once a week': 1,
        'About once a week': 2,
        'Every 4-5 days': 3,
        'Every 2-3 days': 4,
        'At least once every day': 5,
        'Several times every day': 6
    }
    return mapping.get(x, np.nan)

for question in smart_meter_questions:
    df_survey[f'{question}_encoded'] = df_survey[question].apply(map_frequency)

df_survey['smart_meter_interaction_score'] = df_survey[[f'{q}_encoded' for q in smart_meter_questions]].mean(axis=1)
merge_columns.append('smart_meter_interaction_score')

# Q130-Q131: Process interest in renewable energy and microgeneration
interest_questions = {'Q130': 'renewable_energy', 'Q131': 'microgeneration'}

def map_interest(x):
    mapping = {
        'Not at all interested': 0,
        'Not very interested': 1,
        "don't know/haven't thought about it": 2,
        'Fairly interested': 3,
        'Very interested': 4
    }
    return mapping.get(x, np.nan)

for question, interest_type in interest_questions.items():
    col_name = f'interest_in_{interest_type}'
    df_survey[col_name] = df_survey[question].apply(map_interest)
    merge_columns.append(col_name)

# Q132: Process concern about climate change
climate_concern_map = {
    'Not at all concerned': 0,
    'Not very concerned': 1,
    "don't know/No opinion": 2,
    'Fairly concerned': 3,
    'Very concerned': 4
}

df_survey['climate_change_concern'] = df_survey['Q132'].map(climate_concern_map)
merge_columns.append('climate_change_concern')

# Q133: Process lifestyle and environment statement
lifestyle_environment_map = {
    'I\'d like to do a lot more to help the environment': 2,
    'I\'d like to do a bit more to help the environment': 1,
    'I\'m happy with what I do at the moment': 0
}
df_survey['lifestyle_environment'] = df_survey['Q133'].map(lifestyle_environment_map)
merge_columns.append('lifestyle_environment')

# Q134-Q135: Process helpfulness of smart meter for understanding bill and consumption
def map_helpfulness(x):
    if pd.isna(x):
        return np.nan
    mapping = {
        'Not at all helpful': 0,
        'Not very helpful': 1,
        'Have not thought about it': 2,
        'don\'t know': 2,
        'Fairly helpful': 3,
        'Very helpful': 4
    }
    return mapping.get(x, np.nan)

df_survey['smart_meter_bill_understanding'] = df_survey['Q134'].apply(map_helpfulness)
df_survey['smart_meter_consumption_understanding'] = df_survey['Q135'].apply(map_helpfulness)
merge_columns.extend(['smart_meter_bill_understanding', 'smart_meter_consumption_understanding'])

In [8]:
# Import the output file from part 3 (attitudes survey)
df_consumption = pd.read_csv('3) household_energy_consumption_with_attitudinal_features.csv')

# Merge with the energy consumption data
df_merged = pd.merge(df_consumption, df_survey[merge_columns], 
                    left_on='household_id', right_on='Household_id', how='inner')

# Drop the redundant Household_id column
df_merged = df_merged.drop('Household_id', axis=1)

### We will now create a final dataset that has consumption data, tariff data, weather data, as well as household data.

In [9]:
# Load the weather data to merge it with df_merged
df_weather = pd.read_csv('1b) merged_weather_data.csv')

# Print information before merging
print("Before merging:")
print(f"Survey data shape: {df_merged.shape}")
print(f"Weather data shape: {df_weather.shape}")
print(f"Unique timestamps in survey data: {df_merged['timestamp'].nunique()}")
print(f"Unique timestamps in weather data: {df_weather['datetime'].nunique()}")

# Merge the dataframes
df_final_merged = pd.merge(df_merged, df_weather, 
                     left_on='timestamp', 
                     right_on='datetime', 
                     how='inner')

# Drop the redundant datetime column
df_final_merged = df_final_merged.drop('datetime', axis=1)

# Print information after merging
print("\nAfter merging:")
print(f"Merged data shape: {df_final_merged.shape}")
print(f"Unique timestamps in merged data: {df_final_merged['timestamp'].nunique()}")

# Print data types of the merged dataframe
pd.set_option('display.max_rows', None)
print(df_final_merged.dtypes)
pd.reset_option('display.max_rows')

# Export the final merged dataframe to a CSV file
df_final_merged.to_csv('4) final_merged_data.csv', index=False)
print("\nFinal merged dataframe has been exported to '4) final_merged_data.csv'")

Before merging:
Survey data shape: (5326080, 72)
Weather data shape: (8760, 6)
Unique timestamps in survey data: 8760
Unique timestamps in weather data: 8760

After merging:
Merged data shape: (5325472, 77)
Unique timestamps in merged data: 8759
timestamp                                        object
household_id                                     object
consumption                                     float64
tariff                                          float64
washing_machine_fixed_schedule                  float64
tumble_dryer_fixed_schedule                     float64
dishwasher_fixed_schedule                       float64
immersion_water_heater_fixed_schedule           float64
electric_oven_fixed_schedule                    float64
electric_hob_fixed_schedule                     float64
ironing_fixed_schedule                          float64
electric_shower_fixed_schedule                  float64
kettle_fixed_schedule                           float64
lighting_fixed_schedule   

### Both datasets, before merging, had the same number of timestamps, let's look at why we lost one timestamp after merging.

In [10]:
survey_timestamps = set(df_merged['timestamp'])
weather_timestamps = set(df_weather['datetime'])

difference_in_timestamps = survey_timestamps.symmetric_difference(weather_timestamps)

if difference_in_timestamps:
    print(f"Difference in timestamps: {difference_in_timestamps}")
else:
    print("No differences in timestamps between survey and weather data.")

Difference in timestamps: {'2013-01-01 00:00:00', '2014-01-01 00:00:00'}


### It seems each dataset had a unique timestamp, which is not a problem. We now have a merged dataset whose timestamp starts at 2013-1-1 01:00:00 and ends at 2013-12-31 23:00:00

In [11]:
# Check the first and last timestamp of the merged dataset
first_timestamp = df_final_merged['timestamp'].min()
last_timestamp = df_final_merged['timestamp'].max()

print(f"First timestamp in merged dataset: {first_timestamp}")
print(f"Last timestamp in merged dataset: {last_timestamp}")

First timestamp in merged dataset: 2013-01-01 01:00:00
Last timestamp in merged dataset: 2013-12-31 23:00:00
